In [ ]:
#divergencias
import util
from backtesting import Backtest
import estrategias as est

#lista = lista= util.lista_de_monedas ()

lista = ['ETHUSDT','BCHUSDT','XRPUSDT','ADAUSDT','FILUSDT','MATICUSDT','OPUSDT','FETUSDT','BTCUSDT'] # lista acotada

lista=['XRPUSDT']

timeframe = '15m'

###
razon_max = 0.0
razon = 0.0
suma = 0.0
trades = 0
length_sma_max = 0
order_max = 0
length_adx_max = 0
for length_sma in [50]: #range(10,300,20):
    print(f'length_sma: {length_sma}')    
    for order in [550]: #range(200,3000,50):
        print(f'order: {order}')    
        for length_adx in [90]: #range(270,300,10):
            print(f'length_adx: {length_adx}')   
            try:
                suma = 0.0
                trades = 0
                razon = 0.0
                for symbol in lista:
                    try:                
                        #data = est.estrategia_adx (symbol,timeframe,length_sma,order,length_adx,start_date='2023-09-01')
                        data = est.estrategia_martillo (symbol,timeframe='15m',start_date='2024-10-01')
                        bt = Backtest(data,util.backtesting_config,cash=1000000,exclusive_orders=True)
                        output = bt.run()
                        if len(lista)==1:
                            bt.plot(filename="graficos/"+symbol)
                        print(f"Symbol: {symbol} - Return [%]: {round(output['Return [%]'],2)} - # Trades: {output['# Trades']} - Win Rate [%]: {output['Win Rate [%]']}")
                        suma=suma+output['Return [%]']
                        trades=trades+output['# Trades']
                    except Exception as e:
                        print(f"Error en {symbol}: {e}")
                if trades>0:
                    razon = suma/trades
                if razon > razon_max:
                    razon_max = razon
                    length_sma_max = length_sma
                    order_max = order
                    length_adx_max = length_adx
                print(f"Suma total: {suma} - Trades total: {trades} - razón: {razon} - order: {order} - length_adx: {length_adx}")
                print(f"razon_max: {razon_max} - length_sma_max: {length_sma_max} - order_max: {order_max} - length_adx_max: {length_adx_max}")
            except Exception as e:
                print(f"Error en {symbol}: {e}")
                print(f"Suma total: {suma} - Trades total: {trades} - razón: {razon} - order: {order} - length_adx: {length_adx}")
                print(f"razon_max: {razon_max} - length_max: {length_sma_max} - order_max: {order_max} - length_adx_max: {length_adx_max}")
print(f"razon_max: {razon_max} - length_sma_max: {length_sma_max} - order_max: {order_max} - length_adx_max: {length_adx_max}")

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Inicializa el WebDriver
driver = webdriver.Chrome()

# Navega a la página de estadísticas
url = 'https://fantasy.premierleague.com/statistics'
driver.get(url)

# Espera a que la ventana emergente de cookies esté visible y haz clic en "Accept All Cookies"
try:
    accept_cookies_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))
    )
    accept_cookies_button.click()
except Exception as e:
    print(f"No se pudo aceptar las cookies: {e}")

# Espera a que la página cargue completamente
time.sleep(5)

# Inicializa una lista para almacenar la información de los jugadores
player_data = {}

# Función para extraer datos de jugadores en la página actual
def scrape_page():
    time.sleep(2)
    tbody = driver.find_element(By.TAG_NAME, 'tbody')
    rows = tbody.find_elements(By.TAG_NAME, 'tr')

    for row in rows:
        player_button = row.find_element(By.CLASS_NAME, 'ElementInTable__Name-sc-y9xi40-1')
        player_name = player_button.text
        player_button.click()
        time.sleep(2)
        
        dialog = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'root-dialog')))
        info_dialog = dialog.text
        player_data[player_name] = info_dialog

        lines = player_data[player_name].split('\n')

        # Extrae los datos principales
        dict = {
            'posicion': lines[1],
            'nombre_completo': lines[2],
            'equipo': lines[3],
            'precio': lines[5],
            'Form': lines[8],
            'Pts / Match': lines[10],
            'GW8 Pts': lines[13],
            'Total Pts': lines[15],
            'Total Bonus': lines[17],
            'ICT Index': lines[19],
            'TSB%': lines[22],
            'info': lines[1],
        }

        # Procesar estadísticas adicionales
        for i in lines:
            if '  Totals   ' in i:
                break

        campos = ['Pts', 'ST', 'MP', 'GS', 'A', 'xG', 'xA', 'xGI', 'CS', 'GC', 'xGC', 'OG', 'PS', 'PM', 'YC', 'RC', 'S', 'BP', 'BPS', 'I', 'C', 'T', 'II', 'NT', 'SB', '£']
        datos = i.replace('  Totals   ', '').replace('-', '').replace(' ', ',').strip()
        datos_list = [float(x) if x else None for x in datos.split(',')]

        dict2 = {}
        for i in range(0, len(campos) - 1, 1):
            dict2[campos[i]] = datos_list[i]

        dict['info'] = dict2
        player_data[player_name] = dict

        try:
            # Cierra el diálogo del jugador
            close_button = WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.Dialog__CloseButtonWrap-sc-5bogmv-0 button.Dialog__CloseButton-sc-5bogmv-1'))
            )
            close_button.click()
        except Exception as e:
            print(f"No se pudo cerrar el diálogo para {player_name}: {e}")

# Función para verificar si existe el botón "Next" y avanzar de página
def next_page():
    try:
        # Encuentra el botón de 'Siguiente' y haz clic
        next_button = driver.find_element(By.XPATH, "//span[text()='Next']/..")
        next_button.click()
        time.sleep(5)  # Espera a que la nueva página cargue
        return True
    except:
        return False

# Recorre todas las páginas de jugadores
while True:
    scrape_page()
    if not next_page():
        break

# Cierra el navegador
driver.quit()

In [4]:
player_data

{'M.Salah': {'posicion': 'Midfielder',
  'nombre_completo': 'Mohamed Salah',
  'equipo': 'Liverpool',
  'precio': '£12.6m',
  'Form': '7.2',
  'Pts / Match': '9.0',
  'GW8 Pts': '10',
  'Total Pts': '72',
  'Total Bonus': '11',
  'ICT Index': '86.2',
  'TSB%': '38.0%'},
 'Palmer': {'posicion': 'Midfielder',
  'nombre_completo': 'Cole Palmer',
  'equipo': 'Chelsea',
  'precio': '£10.8m',
  'Form': '10.2',
  'Pts / Match': '8.6',
  'GW8 Pts': '2',
  'Total Pts': '69',
  'Total Bonus': '9',
  'ICT Index': '86.9',
  'TSB%': '50.9%'},
 'Haaland': {'posicion': 'Forward',
  'nombre_completo': 'Erling Haaland',
  'equipo': 'Man City',
  'precio': '£15.4m',
  'Form': '3.8',
  'Pts / Match': '8.6',
  'GW8 Pts': '2',
  'Total Pts': '69',
  'Total Bonus': '14',
  'ICT Index': '91.8',
  'TSB%': '73.7%'},
 'Luis Díaz': {'posicion': 'Midfielder',
  'nombre_completo': 'Luis Díaz',
  'equipo': 'Liverpool',
  'precio': '£8.0m',
  'Form': '5.0',
  'Pts / Match': '6.9',
  'GW8 Pts': '1',
  'Total Pts': '5